# Regression Week 5: Ass 2: LASSO (coordinate descent)
## pandas version

see also https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/

also http://scikit-learn.org/stable/

https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/linear_model/coordinate_descent.py

http://au.mathworks.com/help/stats/lasso.html

see solution also at https://github.com/anindya-saha/Machine-Learning-with-Python/blob/master/Coursera-Machine-Learning-Regression/week-5-lasso-assignment-2-graphlab.ipynb  and
https://github.com/kgrodzicki/machine-learning-specialization/blob/master/course-2-regression/notebooks/week-5-lasso-assignment-2-blank.ipynb

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [54]:
import pandas as pd
import numpy as np # note this allows us to refer to numpy as np instead 

# Load in house sales data

**1.** Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [55]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 
              'sqft_living15':float, 'grade':int, 'yr_renovated':int, 
              'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 
              'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 
              'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv('../Week3/kc_house_data.csv', dtype=dtype_dict)

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

print "\nFeatures are ",sales.columns
print "\nNumber of features = %d" %len(sales.columns)
print sales.head(2)


Features are  Index([u'id', u'date', u'price', u'bedrooms', u'bathrooms', u'sqft_living',
       u'sqft_lot', u'floors', u'waterfront', u'view', u'condition', u'grade',
       u'sqft_above', u'sqft_basement', u'yr_built', u'yr_renovated',
       u'zipcode', u'lat', u'long', u'sqft_living15', u'sqft_lot15'],
      dtype='object')

Number of features = 21
           id             date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000  221900.0       3.0       1.00       1180.0   
1  6414100192  20141209T000000  538000.0       3.0       2.25       2570.0   

   sqft_lot  floors  waterfront  view     ...      grade  sqft_above  \
0      5650       1           0     0     ...          7        1180   
1      7242       2           0     0     ...          7        2170   

   sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
0              0      1955             0    98178  47.5112 -122.257   
1            400      1951          1991    98125

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the dataframes. For this notebook, however, we will work with the existing features.

**2.** If you’re using Python: to do the matrix operations required to perform a coordinate descent we will be using the popular python library ‘numpy’ which is a computational library specialized for operations on arrays

# Import useful functions from previous notebook

**3.** Create function ‘get_numpy_data’ (or equivalent) that takes a data set, a list of features (e.g. [‘sqft_living’, ‘bedrooms’]), to be used as inputs, and a name of the output (e.g. ‘price’). This function returns a ‘feature_matrix’ (2D array) consisting of first a column of ones followed by columns containing the values of the input features in the data set in the same order as the input list. It also returns an ‘output_array’ which is an array of the values of the output in the data set (e.g. ‘price’). So we convert the pandas dataframe into a 2D Numpy array.

In [56]:
# inputs
# a dataframe , a list of features (e.g. [‘sqft_living’, ‘bedrooms’]), and target/response (e.g. ‘price’)
# ouputs
#  ‘feature_matrix’ (2D array containing values of features only - the X matrix)
#  an ‘output_array’ ,values of the output/response/target in the data set or y (e.g. ‘price’)

# Called like this, returning data in a tuple
# (example_features, example_output) =  get_numpy_data(sales, ['sqft_living'], 'price')
# NB., sales is the pdFrame, features_list = ['sqft_living', ....], response = 'price')
    
def get_numpy_data(data_pdframe, features, output):
    
    data_pdframe['constant'] = 1 # this is how you add a new column to an dataframe
    
    # add the column 'constant' to the FRONT of the features list 
    # so that we can extract it along with the others:
    features = ['constant'] + features # add two lists
    
    #NB features = 'constant' + features  wont work!! - can't add string constant to a list
    
    print "get_numpy_data(): Features", features, "\n"  
    
    # select the columns of data_pdFrame given by the features list 
    # into the pdFrame features_pdframe (now including constant 1 col):
    features_pdframe = data_pdframe[features]
    
    print "get_numpy_data(): print features_pdframe(head(2)\n",features_pdframe.head(2), "\n"  
    #prints the 2 columns n its values, constant and sqft_living
    
    # Convert the pdframe to its Numpy-array representation.
    # feature_matrix = features_sframe.to_numpy()    #if sframe , use .to_numpy()
    feature_matrix = features_pdframe.as_matrix()    #if pdframe ,use .as_matrix() 
    len(feature_matrix)
    print "get_numpy_data(): print features_pdframe.as_matrix()[0:2, :]\n",feature_matrix[0:2, :] , "\n" 
    
    # assign the column of data_pdframe associated with the output to the SArray output_sarray
    output_pandas_series = data_pdframe[output]
    print "get_numpy_data(): data_pdframe['price']\n", output_pandas_series .head(2), "\n"
    
    # Convert the pdframe column - a series to its Numpy-array representation.
    # output_array =      output_sarray.to_numpy()   #if column is sframe sarray
    output_array = output_pandas_series.as_matrix()  #if column is pandas series
    print "get_numpy_data(): data_pdframe['price'].as_matrix()\n", output_array[0:2], "\n"
    
    return(feature_matrix, output_array)

**4.** Define `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights. This function accepts a 2D array ‘feature_matrix’ and a 1D array ‘weights’ and return a 1D array ‘predictions’.

In [57]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns 
    # and weights is a corresponding numpy array (our list of model coefficients/weights)
    # create the predictions vector by using np.dot()
    
    predictions = np.dot(feature_matrix, weights)
    #or         = np.sum(feature_matrix * weights)
    return(predictions)

# Normalize features
**5.** In the house dataset, features vary wildly in their relative magnitude: ** `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`.** This is problematic because **"small" weights are dropped first as `l1_penalty` goes up!!**. 

To give equal considerations for all features, we need to **normalize features**: we divide each feature by its 2-norm so that the transformed feature has norm 1.

We can do this normalization easily with Numpy: let us first consider a small matrix.

In [58]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [59]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print np.sqrt(3**2 + 4**2), np.sqrt(5**2 + 12**2),np.sqrt(8**2 + 15**2)
print norms
norms1 = np.linalg.norm(X, axis=1) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print np.sqrt(3**2 + 5**2 + 8**2), np.sqrt(4**2 + 12**2 + 15**2)
print norms1

5.0 13.0 17.0
[  5.  13.  17.]
9.89949493661 19.6214168703
[  9.89949494  19.62141687]


To normalize, apply element-wise division:

In [60]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


**6.** Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the *norms of original features*. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [61]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix/ norms
    
    return  (normalized_features, norms)

To test the function, run the following:

In [62]:
(features, norms) = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

**7. We seek to obtain a sparse set of weights by minimizing the LASSO cost function**
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times (round robin style).  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

**8.** For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates or features 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

**9.** Consider a simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. The output is ‘price’.

* First, run get_numpy_data() (or equivalent) to obtain a feature matrix with 3 columns (constant column added). Use the entire ‘sales’ dataset for now.
* Normalize columns of the feature matrix. Save the norms of original features as ‘norms’.
* Set initial weights to [1,4,1].
* Make predictions with feature matrix and initial weights.
* Compute values of ro[i], where

`ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]`

Run get_numpy_data() (or equivalent) to obtain a feature matrix with 3 columns (constant column added). 

In [63]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
print len(output)

get_numpy_data(): Features ['constant', 'sqft_living', 'bedrooms'] 

get_numpy_data(): print features_pdframe(head(2)
   constant  sqft_living  bedrooms
0         1       1180.0       3.0
1         1       2570.0       3.0 

get_numpy_data(): print features_pdframe.as_matrix()[0:2, :]
[[  1.00000000e+00   1.18000000e+03   3.00000000e+00]
 [  1.00000000e+00   2.57000000e+03   3.00000000e+00]] 

get_numpy_data(): data_pdframe['price']
0    221900.0
1    538000.0
Name: price, dtype: float64 

get_numpy_data(): data_pdframe['price'].as_matrix()
[ 221900.  538000.] 

21613


Normalize columns of the feature matrix.

In [64]:
(simple_feature_matrix, norms) = normalize_features(simple_feature_matrix)
simple_feature_matrix

array([[ 0.00680209,  0.00353021,  0.00583571],
       [ 0.00680209,  0.00768869,  0.00583571],
       [ 0.00680209,  0.00230361,  0.00389048],
       ..., 
       [ 0.00680209,  0.00305154,  0.00389048],
       [ 0.00680209,  0.00478673,  0.00583571],
       [ 0.00680209,  0.00305154,  0.00389048]])

We assign some random set of initial weights [1,4,1] and inspect the values of `ro[i]`:

In [65]:
weights = np.array([1., 4., 1.])
weights

array([ 1.,  4.,  1.])

Use `predict_output()` to make predictions with feature matrix and initial weights.

In [66]:
prediction = predict_output(simple_feature_matrix, weights)
# its simply np.dot(simple_feature_matrix, weights)
prediction


array([ 0.02675867,  0.04339256,  0.01990703, ...,  0.02289873,
        0.03178473,  0.02289873])

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[  [feature_i]* ( output - prediction + w[i]*[feature_i] ) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [67]:
simple_feature_matrix[:,0]   #array([ 1.,  1.,  1., ...,  1.,  1.,  1.])
simple_feature_matrix[:,1]

array([ 0.00353021,  0.00768869,  0.00230361, ...,  0.00305154,
        0.00478673,  0.00305154])

In [68]:
ro = {}   #or   ro = list()

print len(weights)
for i in xrange(len(weights)): # loop over each weight
    feature_i = simple_feature_matrix[:,i]
    #ro[i]= np.sum([  [feature_i] * (output - prediction + weights[i]*[feature_i]) ])
    ro[i] =  np.dot(  (feature_i),  (output - prediction + weights[i] *feature_i))
    #ro.append(ro_i)

print ro
# my way of doing ro_i gives these ro's, 1st and last is same as  mine!!!
#79400300.0349
#351757871.092
#80966698.676

3
{0: 79400300.014522895, 1: 87939470.823251754, 2: 80966698.66623947}


***QUIZ QUESTION***

**10.** Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [69]:
# note lambda = l1_penalty = ro[i] * 2
# w[1] = 0 if -lambda/2 <= ro[1] <= lambda/2
# w[2] = 0 if -lambda/2 <= ro[2] <= lambda/2
#  so the range of values of l1_penalty that would set w[1] zero
print("%s <= 2 * ro[1] <= %s" % (-2 * ro[1], 2 * ro[1]))
#  so the range of values of l1_penalty that would set w[2] zero
print("%s <= 2 * ro[2] <= %s" % (-2 * ro[2], 2 * ro[2]))

-175878941.647 <= 2 * ro[1] <= 175878941.647
-161933397.332 <= 2 * ro[2] <= 161933397.332


***QUIZ QUESTION***

**11.** What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

We can see from above that l1_penalty range for ro[2] is subset of ro[2] peanlty, so for any lambda falling in range for ro[2], wud set both w[1] and w[2] to zero. 

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

**12.** Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [70]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    feature_i = feature_matrix[:,i]
    ro_i =       (feature_i * (output - prediction + weights[i] * feature_i)).sum()
    #ro_i = np.sum(feature_i * (output - prediction + weights[i] * feature_i))  
    #ro_i = np.dot(feature_i,  (output - prediction + weights[i] * feature_i))
    # all three ways above are equivalent
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.0:
        new_weight_i = ro_i + l1_penalty/2.0
    elif ro_i > l1_penalty/2.0:
        new_weight_i = ro_i - l1_penalty/2.0
    else:
        new_weight_i = 0.0
    
    return new_weight_i

To test the function, run the following cell:

In [71]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, 
                                    np.array([[3./math.sqrt(13),
                                               1./math.sqrt(10)],
                                              [2./math.sqrt(13),
                                               3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

**13.** Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights


The function should accept the following parameters:

*    Feature matrix
*    Output array
*    Initial weights
*    L1 penalty
*    Tolerance

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [72]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    
    converged = False
    weights = np.array(initial_weights.copy()) # make sure it's a numpy array
    
    while not converged:
        change = list()
        for i in range(len(weights)):
            old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
            # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
            # and old values for weight[i], ..., weight[d-1]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)

            # use old_weights_i to compute change in coordinate
            change_in_weight = abs(old_weights_i - weights[i])
            change.append(change_in_weight)
                       
        if max(change) < tolerance:
            converged = True
                
    return weights    

**14.** Let us now go back to the simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. Using ‘get_numpy_data’ (or equivalent), extract the feature matrix and the output array from from the house dataframe. Then normalize the feature matrix using ‘normalized_features()’ function.

Using the following parameters, learn the weights on the sales dataset.

*    Initial weights = all zeros
*    L1 penalty = 1e7
*    Tolerance = 1.0

In [73]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [74]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

get_numpy_data(): Features ['constant', 'sqft_living', 'bedrooms'] 

get_numpy_data(): print features_pdframe(head(2)
   constant  sqft_living  bedrooms
0         1       1180.0       3.0
1         1       2570.0       3.0 

get_numpy_data(): print features_pdframe.as_matrix()[0:2, :]
[[  1.00000000e+00   1.18000000e+03   3.00000000e+00]
 [  1.00000000e+00   2.57000000e+03   3.00000000e+00]] 

get_numpy_data(): data_pdframe['price']
0    221900.0
1    538000.0
Name: price, dtype: float64 

get_numpy_data(): data_pdframe['price'].as_matrix()
[ 221900.  538000.] 



Then, run your implementation of LASSO coordinate descent:

In [75]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
weights

array([ 21624997.95951909,  63157247.20788956,         0.        ])

**QUIZ QUESTIONS**

**15. What is the RSS of the learned model on the normalized dataset?**

**16. Which features had weight zero at convergence?**

In [76]:
def get_residual_sum_of_squares(predictions, output):
     
    # Then compute the residuals/errors
    residual = output - predictions

    # Then square and add them up
    residual_squared = residual * residual
    
    RSS = residual_squared.sum()
    print "RSS", RSS
    return(RSS)

In [77]:
print(weights)

predictions = predict_output(normalized_simple_feature_matrix, weights)
RSS = get_residual_sum_of_squares(predictions, output)
print("\nRSS: $%.6f" % (RSS))

[ 21624997.95951909  63157247.20788956         0.        ]
RSS 1.63049247672e+15

RSS: $1630492476715386.500000


# Evaluating LASSO fit with more features

**17.** Let us split the sales dataset into training and test sets. If you are using GraphLab Create, call ‘random_split’ with .8 ratio and seed=0. Otherwise, please down the corresponding csv files from the downloads section.

In [78]:
train_data = pd.read_csv('../Week3/wk3_kc_house_train_data.csv', dtype=dtype_dict)
test_data = pd.read_csv('../Week3/wk3_kc_house_test_data.csv', dtype=dtype_dict)

Let us consider the following set of features.

In [79]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']
my_output = 'price'

**18.** First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [80]:
(all_feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
(normalized_all_feature_matrix, all_norms) = normalize_features(all_feature_matrix) # normalize features

get_numpy_data(): Features ['constant', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated'] 

get_numpy_data(): print features_pdframe(head(2)
   constant  bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  \
0         1       4.0        3.0       1960.0      5000     1.0           0   
1         1       4.0        4.5       5420.0    101930     1.0           0   

   view  condition  grade  sqft_above  sqft_basement  yr_built  yr_renovated  
0     0          5      7        1050            910      1965             0  
1     0          3     11        3890           1530      2001             0   

get_numpy_data(): print features_pdframe.as_matrix()[0:2, :]
[[  1.00000000e+00   4.00000000e+00   3.00000000e+00   1.96000000e+03
    5.00000000e+03   1.00000000e+00   0.00000000e+00   0.00000000e+00
    5.00000000e+00   7.00000000e+00   1.05000000e+03   9.10000000e

**19.** First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [81]:
initial_weights = np.zeros(14)
l1_penalty = 1e7
tolerance = 1.0

weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [82]:
# function to show only non-zero weights
def show_nnz_features(w, f):    
    for i in range(len(w)):
        if i == 0:
            continue
        weight = w[i]
        if weight != 0.:
            print f[i - 1]

***QUIZ QUESTION***

**20.What features had non-zero weight in this case?**

In [83]:
print(zip(['constant'] + all_features, weights1e7))

[('constant', 23864692.509538405), ('bedrooms', 0.0), ('bathrooms', 0.0), ('sqft_living', 30495548.132547192), ('sqft_lot', 0.0), ('floors', 0.0), ('waterfront', 1901633.614755936), ('view', 5705765.0167326573), ('condition', 0.0), ('grade', 0.0), ('sqft_above', 0.0), ('sqft_basement', 0.0), ('yr_built', 0.0), ('yr_renovated', 0.0)]


In [84]:
show_nnz_features(weights1e7, all_features)             

sqft_living
waterfront
view


**21.** Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [85]:
initial_weights = np.zeros(14)
l1_penalty = 1e8
tolerance = 1.0

weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

***QUIZ QUESTION***

**22. What features had non-zero weight in this case?**

In [86]:
print(zip(['constant'] + all_features, weights1e8))

[('constant', 53621004.689714715), ('bedrooms', 0.0), ('bathrooms', 0.0), ('sqft_living', 0.0), ('sqft_lot', 0.0), ('floors', 0.0), ('waterfront', 0.0), ('view', 0.0), ('condition', 0.0), ('grade', 0.0), ('sqft_above', 0.0), ('sqft_basement', 0.0), ('yr_built', 0.0), ('yr_renovated', 0.0)]


In [87]:
show_nnz_features(weights1e8, all_features)    # shows nothins - so all zero except intercept

**23.** Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [88]:
initial_weights = np.zeros(14)
l1_penalty = 1e4
tolerance = 5e5

weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [89]:
show_nnz_features(weights1e4, all_features)  

bedrooms
bathrooms
sqft_living
sqft_lot
floors
waterfront
view
condition
grade
sqft_above
sqft_basement
yr_built
yr_renovated


***QUIZ QUESTION***

**24. What features had non-zero weight in this case?** About 13 features out of 21 - see above

In [90]:
print(zip(['constant'] + all_features, weights1e4))

[('constant', 57481091.133020647), ('bedrooms', -13652628.5402242), ('bathrooms', 12462713.071262423), ('sqft_living', 57942788.373312257), ('sqft_lot', -1475769.6942756311), ('floors', -4904547.7554655103), ('waterfront', 5349050.1863616901), ('view', 5845253.5621363353), ('condition', -416038.96981262043), ('grade', 2682274.5948850354), ('sqft_above', 242649.68555066799), ('sqft_basement', -1285549.6676812347), ('yr_built', -54779474.227683559), ('yr_renovated', 2167703.0661023338)]


## Rescaling learned weights

**25.** Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

**26.** Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [91]:
normalized_weights1e4 = weights1e4 / all_norms 
normalized_weights1e7 = weights1e7 / all_norms 
normalized_weights1e8 = weights1e8 / all_norms

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

In [92]:
print normalized_weights1e7[3]

135.176516461


## Evaluating each of the learned models on the test data

**27.** Let's now evaluate the three models on the test data. Extract the feature matrix and output array from the TEST set. But this time, do NOT normalize the feature matrix. Instead, use the normalized version of weights to make predictions.

In [93]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

get_numpy_data(): Features ['constant', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated'] 

get_numpy_data(): print features_pdframe(head(2)
   constant  bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  \
0         1       3.0        2.5       1890.0      6560     2.0           0   
1         1       5.0        2.0       1810.0      4850     1.5           0   

   view  condition  grade  sqft_above  sqft_basement  yr_built  yr_renovated  
0     0          3      7        1890              0      2003             0  
1     0          3      7        1810              0      1900             0   

get_numpy_data(): print features_pdframe.as_matrix()[0:2, :]
[[  1.00000000e+00   3.00000000e+00   2.50000000e+00   1.89000000e+03
    6.56000000e+03   2.00000000e+00   0.00000000e+00   0.00000000e+00
    3.00000000e+00   7.00000000e+00   1.89000000e+03   0.00000000e

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [94]:
predictions = predict_output(test_feature_matrix, normalized_weights1e4)
RSS = get_residual_sum_of_squares(predictions, test_output)
print("RSS on TEST data with normalized_weights1e4: $%.6f" % (RSS))

RSS 1.29085259902e+14
RSS on TEST data with normalized_weights1e4: $129085259901909.140625


In [95]:
predictions = predict_output(test_feature_matrix, normalized_weights1e7)
RSS = get_residual_sum_of_squares(predictions, test_output)
print("RSS on TEST data with normalized_weights1e7: $%.6f" % (RSS))

RSS 1.63103564165e+14
RSS on TEST data with normalized_weights1e7: $163103564164999.125000


In [96]:
predictions = predict_output(test_feature_matrix, normalized_weights1e8)
RSS = get_residual_sum_of_squares(predictions, test_output)
print("RSS on TEST data with normalized_weights1e8: $%.6f" % (RSS))

RSS 2.8471892521e+14
RSS on TEST data with normalized_weights1e8: $284718925209874.000000


***QUIZ QUESTION***

Which model performed best on the test data? The one which had smallest lambda or l1_penalty=1e4, because this model allowed most flexible fit (13 features out 0f 21 had non zero weights). So by providing lowest cost/penalty to overfitting, most coefficients/weights were non-zero, allowing to fit data best,,,hence lowest bias and because rss ~ mse  = bias^2 + variance, this model gave lowest rss. On the other hand, with a very large L1 penalty of 1e8, all coeficients or weights were zero, except the intercept. So out of 21 features, non had non zero weights. So worst fit to data and hence BIAS^2 would be very very high (even though VAR would also be small), but because of BIAS^2 contribution to RSS, this model has the highest RSS.